In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. 시드 값 고정 및 GPU 장비 설정
  
- 개발 환경: colab pro
- 사용 프레임워크: pytorch

### 시드값 고정

In [28]:
import torch # 파이토치
import torchvision
import random
import numpy as np
import pandas as pd
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm


# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### GPU 장비 설정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. 데이터 준비

In [4]:
main_path = '/content/drive/MyDrive/data/programmers_ai/mlops/COVID_19_XRAY/'

# 훈련, 검증, 테스트 경로
train_path = main_path + 'train/'
valid_path = main_path + 'val/'
test_path = main_path + 'test/'

### 데이터 증강을 위한 이미지 변환기 정의

In [5]:
# 훈련 데이터용 변환기
transform_train = transforms.Compose([
    transforms.Resize((224, 224)), # 이미지 크기 조절 for resnet
    transforms.CenterCrop(180),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
    transforms.Resize((224, 224)), # 이미지 크기 조절 for resnet
    transforms.CenterCrop(180),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

### 데이터셋 및 데이터 로더 생성

In [6]:
# 훈련 데이터셋
datasets_train = ImageFolder(root=train_path, transform=transform_train)

# 검증 데이터셋
datasets_valid = ImageFolder(root=valid_path, transform=transform_test)

In [7]:
def seed_worker(workder_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# 제너레이터 시드값 고정
g = torch.Generator()
g.manual_seed(0)

In [8]:
batch_size = 8

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

## 모델 생성

- https://pytorch.kr/hub/pytorch_vision_resnet/
- https://yeong-jin-data-blog.tistory.com/entry/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98-%EC%8A%A4%ED%84%B0%EB%94%94-%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5-%EB%AA%A8%EB%8D%B8-%ED%94%84%EB%A6%AC%EC%A7%95

In [9]:
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [10]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [11]:
# 모델 구조 수정
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

In [12]:
print('모델 파라미터 개수 :', sum(param.numel() for param in model.parameters()))

모델 파라미터 개수 : 11169922


## 모델 훈련 및 성능 검증

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### 훈련 함수 작성

In [22]:
def train(model, loader_train, loader_valid, criterion, optimizer, scheduler=None, epochs=10, save_file='model_state_dict.pth'):

  valid_loss_min = np.inf # 최소 손실값 초기화 (검증 데이터용) 
  for epoch in range(epochs):
    print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
    # == [ 훈련 ] ==============================================
    model.train() # 모델 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화(훈련 데이터용)
    for images, labels in tqdm(loader_train):
      # 이미지, 레이블 데이터 미니배치를 장비에 할당
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # 옵티마이저 내 기울기 초기화
      outputs = model(images) # 순전파: 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
      loss = criterion(outputs, labels) # 손실 함수를 활용해 outputs와 labels의 손실값 계산
      epoch_train_loss += loss.item() # 현재 배치에서의 손실 추가(훈련 데이터용)
      loss.backward() # 역전파 수행
      optimizer.step() # 가중치 갱신
      if scheduler != None: # 스케줄러 학습률 갱신
        scheduler.step()
    # 훈련 데이터 손실값 출력
    print(f'\t훈련 데이터 손실값: {epoch_train_loss/len(loader_train): .4f}')

    # == [ 검증 ] ==============================================
    model.eval()         # 모델을 평가 상태로 설정 
    epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
    preds_list = []      # 예측값 저장용 리스트 초기화
    true_list = []       # 실젯값 저장용 리스트 초기화
        
    with torch.no_grad(): # 기울기 계산 비활성화
      for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
                
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_valid_loss += loss.item()
                
        # 예측값 및 실제값 
        preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
        true = labels.cpu().numpy() 
        preds_list.extend(preds)
        true_list.extend(true)
                
    # 정확도, 재현율, F1 점수 계산
    val_accuracy = accuracy_score(true_list, preds_list)
    val_recall = recall_score(true_list, preds_list)
    val_f1_score = f1_score(true_list, preds_list)

    # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
    print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
    print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
    # == [ 최적 모델 가중치 찾기 ] ==============================
    # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 
    if epoch_valid_loss <= valid_loss_min: 
        print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
        # 모델 가중치를 파일로 저장 
        torch.save(model.state_dict(), save_file) 
        valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 
  return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환

### 훈련 및 성능 검증

In [23]:
# 모델 훈련
model_state_dict = train(model=model, loader_train=loader_train, loader_valid=loader_valid, criterion=criterion, optimizer=optimizer)

에폭 [1/10] 
-----------------------------


  0%|          | 0/201 [00:00<?, ?it/s]

	훈련 데이터 손실값:  0.6954
	검증 데이터 손실값 : 0.8983
	정확도 : 0.5101 / 재현율 : 0.7500 / F1 점수 : 0.6061
	### 검증 데이터 손실값 감소 (inf --> 44.9164). 모델 저장
에폭 [2/10] 
-----------------------------


  0%|          | 0/201 [00:00<?, ?it/s]

	훈련 데이터 손실값:  0.6654
	검증 데이터 손실값 : 0.6342
	정확도 : 0.6307 / 재현율 : 0.5000 / F1 점수 : 0.5764
	### 검증 데이터 손실값 감소 (44.9164 --> 31.7121). 모델 저장
에폭 [3/10] 
-----------------------------


...

	훈련 데이터 손실값:  0.5803
	검증 데이터 손실값 : 0.6070
	정확도 : 0.6658 / 재현율 : 0.5400 / F1 점수 : 0.6189
	### 검증 데이터 손실값 감소 (30.4133 --> 30.3489). 모델 저장
에폭 [9/10] 
-----------------------------


  0%|          | 0/201 [00:00<?, ?it/s]

	훈련 데이터 손실값:  0.5798
	검증 데이터 손실값 : 0.7222
	정확도 : 0.7060 / 재현율 : 0.5350 / F1 점수 : 0.6465
에폭 [10/10] 
-----------------------------


  0%|          | 0/201 [00:00<?, ?it/s]

	훈련 데이터 손실값:  0.5630
	검증 데이터 손실값 : 0.5388
	정확도 : 0.6884 / 재현율 : 0.5900 / F1 점수 : 0.6556
	### 검증 데이터 손실값 감소 (30.3489 --> 26.9391). 모델 저장


In [24]:
# 최적 가중치 불러오기
model.load_state_dict(model_state_dict)

<All keys matched successfully>

## 예측 및 평가 결과

In [25]:
import cv2
from torch.utils.data import Dataset

In [26]:
# test용 imagedataset

class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id + '.png'
        image = cv2.imread(img_path)
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        return image

In [30]:
path = '/content/drive/MyDrive/data/programmers_ai/mlops/COVID_19_XRAY/'
submission = pd.read_csv(os.path.join(path, 'submission.csv'))
submission.head()

,filename,label
0,image_001.png,NaN
1,image_002.png,NaN
2,image_003.png,NaN
3,image_004.png,NaN
4,image_005.png,NaN


In [32]:
dataset_test = ImageDataset(submission, img_dir=os.path.join(path, 'test/'), transform=transform_test)

In [33]:
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

In [35]:
# 예측
model.eval()
preds = np.zeros((len(submission), 2))

with torch.no_grad():
  for i, images in enumerate(loader_test):
    images = images.to(device)
    outputs = model(images)
    preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
    print(preds_part)

TypeError: ignored